In [105]:
import torch
from einops import einsum,rearrange

In [106]:
theta = 10000
d_k = 64
k = torch.arange(d_k // 2,dtype=torch.float32)
theta  = 1.0 / (theta ** (2 * k / d_k))
theta

tensor([1.0000e+00, 7.4989e-01, 5.6234e-01, 4.2170e-01, 3.1623e-01, 2.3714e-01,
        1.7783e-01, 1.3335e-01, 1.0000e-01, 7.4989e-02, 5.6234e-02, 4.2170e-02,
        3.1623e-02, 2.3714e-02, 1.7783e-02, 1.3335e-02, 1.0000e-02, 7.4989e-03,
        5.6234e-03, 4.2170e-03, 3.1623e-03, 2.3714e-03, 1.7783e-03, 1.3335e-03,
        1.0000e-03, 7.4989e-04, 5.6234e-04, 4.2170e-04, 3.1623e-04, 2.3714e-04,
        1.7783e-04, 1.3335e-04])

In [107]:
torch.manual_seed(4)
x = torch.randn(4, 12, 64)
positions = torch.arange(12,dtype=torch.float)

In [108]:
position_preped = rearrange(positions, " ... -> ... 1")
angle = position_preped * theta
angle[1]

tensor([1.0000e+00, 7.4989e-01, 5.6234e-01, 4.2170e-01, 3.1623e-01, 2.3714e-01,
        1.7783e-01, 1.3335e-01, 1.0000e-01, 7.4989e-02, 5.6234e-02, 4.2170e-02,
        3.1623e-02, 2.3714e-02, 1.7783e-02, 1.3335e-02, 1.0000e-02, 7.4989e-03,
        5.6234e-03, 4.2170e-03, 3.1623e-03, 2.3714e-03, 1.7783e-03, 1.3335e-03,
        1.0000e-03, 7.4989e-04, 5.6234e-04, 4.2170e-04, 3.1623e-04, 2.3714e-04,
        1.7783e-04, 1.3335e-04])

In [109]:
R_cos = torch.cos(angle).unsqueeze(-1)
R_sin = torch.sin(angle).unsqueeze(-1)
R_cos[1][0],R_sin[1][0]

(tensor([0.5403]), tensor([0.8415]))

In [110]:
R = torch.stack([R_cos, -R_sin, R_sin, R_cos], dim=-1).squeeze(-2)
R[1][0]

tensor([ 0.5403, -0.8415,  0.8415,  0.5403])

In [111]:
R_final = rearrange(
    R,
    " ... seq dk_2 (x1 x2) -> ... seq dk_2 x2 x1",
    x1=2,
    x2=2,
)
R_final[1][0]

tensor([[ 0.5403,  0.8415],
        [-0.8415,  0.5403]])

In [112]:
X_preped = rearrange(x, "... seq ( dk_2 y ) -> ... seq dk_2 1 y ", y=2)
X_preped.shape

torch.Size([4, 12, 32, 1, 2])

In [113]:
preres = einsum(
            X_preped,
            R_final,
            " ... seq dk_2 x1 y, ... seq dk_2 y x -> ... seq dk_2 x1 x",
        )
res = rearrange(preres, "... seq dk_2 x y -> ... seq (dk_2 x y)")

In [114]:
res[0,1]

tensor([ 0.1500, -0.0721, -0.1978,  0.5219,  0.3934, -0.1684, -0.8209, -0.0424,
         0.6446,  2.1171,  1.5502,  0.1070, -0.9238, -1.4239,  1.0173,  0.3385,
         0.5653,  2.5857, -0.9497, -0.0881,  0.2142,  0.6893,  1.7383, -0.3313,
         1.0766,  0.4893, -1.2753, -0.4801, -1.0555, -1.0249, -0.6775,  1.5145,
         0.0504, -1.3858,  2.8358, -0.1538, -0.6845, -1.6847, -0.5692, -1.0971,
         0.1646, -0.8997,  0.1821,  0.5116,  0.2960,  1.5625, -2.0234, -0.0756,
        -0.3919,  0.1265, -0.8289, -1.6487,  0.2351, -0.5801, -1.5086,  0.1102,
        -0.2663,  1.0876, -0.9444, -0.7945, -1.2031,  0.6716,  0.3124, -1.6153])